# **UTS**


## Mengambil dataset

In [1]:
import pandas as pd 
import numpy as np
# memangggil dataset 
data = pd.read_csv("https://raw.githubusercontent.com/nuskhatulhaqqi/data_mining/main/dataset-baru.csv")
# menampilkan dataset
data.head()

,Unnamed: 0,Abstrak,Kategori
0,0,Sistem informasi akademik (SIAKAD) merupaka...,RPL
1,1,Berjalannya koneksi jaringan komputer dengan l...,RPL
2,2,Web server adalah sebuah perangkat lunak serve...,RPL
3,3,Penjadwalan kuliah di Perguruan Tinggi me...,KOMPUTASI
4,4,Seiring perkembangan teknologi yang ada diduni...,RPL


## Pre-Prosessing

### Case Folding

In [2]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data['Abstrak'] = data['Abstrak'].str.lower()


print('Case Folding Result : \n')
print(data['Abstrak'].head(5))

Case Folding Result : 

0    sistem  informasi  akademik  (siakad) merupaka...
1    berjalannya koneksi jaringan komputer dengan l...
2    web server adalah sebuah perangkat lunak serve...
3    penjadwalan  kuliah  di  perguruan  tinggi  me...
4    seiring perkembangan teknologi yang ada diduni...
Name: Abstrak, dtype: object


### Tokenizing

In [5]:
# import modul nltk
import nltk
# download paket punkt 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_text_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['Abstrak'] = data['Abstrak'].apply(remove_text_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['Abstrak'] = data['Abstrak'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['Abstrak'] = data['Abstrak'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['Abstrak'] = data['Abstrak'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['Abstrak'] = data['Abstrak'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['Abstrak'] = data['Abstrak'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['text_tokens'] = data['Abstrak'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data['text_tokens'].head())

Tokenizing Result : 

0    [sistem, informasi, akademik, siakad, merupaka...
1    [berjalannya, koneksi, jaringan, komputer, den...
2    [web, server, adalah, sebuah, perangkat, lunak...
3    [penjadwalan, kuliah, di, perguruan, tinggi, m...
4    [seiring, perkembangan, teknologi, yang, ada, ...
Name: text_tokens, dtype: object


### Stopword



In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['text_tokens_WSW'] = data['text_tokens'].apply(stopwords_removal) 


print(data['text_tokens_WSW'].head())

0    [sistem, informasi, akademik, siakad, sistem, ...
1    [berjalannya, koneksi, jaringan, komputer, lan...
2    [web, server, perangkat, lunak, server, berfun...
3    [penjadwalan, kuliah, perguruan, kompleks, per...
4    [seiring, perkembangan, teknologi, didunia, mu...
Name: text_tokens_WSW, dtype: object


### Stemming

In [10]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 6.1 MB/s eta 0:00:00


In [11]:
pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.6 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=561945371f65ab85e9030797480878bec8f8fa40344dbb7c7e5f41c9daee3fa3
  Stored in directory: /root/.cache/pip/wheels/2b/5e/f2/3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter


In [12]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['text_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['text_tokens_stemmed'] = data['text_tokens_WSW'].swifter.apply(get_stemmed_term)
print(data['text_tokens_stemmed'])

Streaming output truncated to the last 5000 lines.
akademiknya : akademik
ilmu : ilmu
dunn : dunn
nem : nem
fisika : fisika
kimia : kimia
geografi : geografi
sosiologi : sosiologi
ppkn : ppkn
agama : agama
conversation : conversation
penjaskes : penjaskes
memetakkan : petak
intensif : intensif
advanced : advanced
smk : smk
satuan : satu
keahlian : ahli
dipadukan : padu
constraint : constraint
satisfaction : satisfaction
problem : problem
kromosom : kromosom
reproduksi : reproduksi
pc : pc
pm : pm
bottomhat : bottomhat
dilasi : dilasi
opening : opening
pembuluh : buluh
retina : retina
fundus : fundus
rgb : rgb
green : green
channel : channel
saturasi : saturasi
contrast : contrast
limited : limited
clahe : clahe
kontras : kontras
dgaussian : dgaussian
memperjelas : jelas
binerisasi : biner
median : median
retinal : retinal
for : for
vessel : vessel
drive : drive
blog : blog
penyumbang : sumbang
notabene : notabene
blogger : blogger
lurus : lurus
pemilahan : pilah
menjelajahi : jajah
exp

Pandas Apply:   0%|          | 0/805 [00:00<?, ?it/s]

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, fungsi...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
800    [investasi, saham, milik, resiko, rugi, gera, ...
801    [information, retrieval, ir, ambil, informasi,...
802    [klasifikasi, citra, proses, kelompok, piksel,...
803    [identifikasi, atribut, pejal, kaki, salah, te...
804    [topik, deteksi, objek, tarik, perhati, kemban...
Name: text_tokens_stemmed, Length: 805, dtype: object


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%cd /content/drive/MyDrive/prosaindata/tugas

/content/drive/MyDrive/prosaindata/tugas


In [17]:
data.to_csv("Text_Preprocessing.csv")

In [18]:
data_pre = pd.read_csv("Text_Preprocessing.csv")
data_pre.head()

,Unnamed: 0.1,Unnamed: 0,Abstrak,Kategori,text_tokens,text_tokens_WSW,text_tokens_stemmed
0,0,0,sistem informasi akademik siakad merupakan sis...,RPL,"['sistem', 'informasi', 'akademik', 'siakad', ...","['sistem', 'informasi', 'akademik', 'siakad', ...","['sistem', 'informasi', 'akademik', 'siakad', ..."
1,1,1,berjalannya koneksi jaringan komputer dengan l...,RPL,"['berjalannya', 'koneksi', 'jaringan', 'komput...","['berjalannya', 'koneksi', 'jaringan', 'komput...","['jalan', 'koneksi', 'jaring', 'komputer', 'la..."
2,2,2,web server adalah sebuah perangkat lunak serve...,RPL,"['web', 'server', 'adalah', 'sebuah', 'perangk...","['web', 'server', 'perangkat', 'lunak', 'serve...","['web', 'server', 'perangkat', 'lunak', 'serve..."
3,3,3,penjadwalan kuliah di perguruan tinggi merupak...,KOMPUTASI,"['penjadwalan', 'kuliah', 'di', 'perguruan', '...","['penjadwalan', 'kuliah', 'perguruan', 'komple...","['jadwal', 'kuliah', 'guru', 'kompleks', 'masa..."
4,4,4,seiring perkembangan teknologi yang ada diduni...,RPL,"['seiring', 'perkembangan', 'teknologi', 'yang...","['seiring', 'perkembangan', 'teknologi', 'didu...","['iring', 'kembang', 'teknologi', 'dunia', 'mu..."


## TF- IDF

### tf idf dengan sklearn

In [19]:
# mengambil data hasil proses pre-prosessing sebelumnya untuk kategori dan text hasil stemming
datap = pd.read_csv("Text_Preprocessing.csv", usecols=["Kategori", "text_tokens_stemmed"])
datap.columns = ["Kategori", "Abstrak"]
datap

,Kategori,Abstrak
0,RPL,"['sistem', 'informasi', 'akademik', 'siakad', ..."
1,RPL,"['jalan', 'koneksi', 'jaring', 'komputer', 'la..."
2,RPL,"['web', 'server', 'perangkat', 'lunak', 'serve..."
3,KOMPUTASI,"['jadwal', 'kuliah', 'guru', 'kompleks', 'masa..."
4,RPL,"['iring', 'kembang', 'teknologi', 'dunia', 'mu..."
...,...,...
800,KOMPUTASI,"['investasi', 'saham', 'milik', 'resiko', 'rug..."
801,KOMPUTASI,"['information', 'retrieval', 'ir', 'ambil', 'i..."
802,KOMPUTASI,"['klasifikasi', 'citra', 'proses', 'kelompok',..."
803,KOMPUTASI,"['identifikasi', 'atribut', 'pejal', 'kaki', '..."


In [20]:
datap.shape

(805, 2)

In [21]:
# join list of token as single document string
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
datap["text_join"] = datap["Abstrak"].apply(join_text_list)

datap["text_join"].head()

0    sistem informasi akademik siakad sistem inform...
1    jalan koneksi jaring komputer lancar ganggu ha...
2    web server perangkat lunak server fungsi terim...
3    jadwal kuliah guru kompleks masalah variabel t...
4    iring kembang teknologi dunia muncul teknologi...
Name: text_join, dtype: object

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# banyaknya term yang akan digunakan, 
# di pilih berdasarkan top max_features 
# yang diurutkan berdasarkan term frequency seluruh corpus
max_features = 100000

# Feature Engineering 
print ("------- TF-IDF data -------")

tf_idf = TfidfVectorizer(max_features=max_features, binary=True)
tfidf_mat = tf_idf.fit_transform(datap["text_join"]).toarray()

print("TF-IDF ", type(tfidf_mat), tfidf_mat.shape)

------- TF-IDF data -------
TF-IDF  <class 'numpy.ndarray'> (805, 6511)


In [23]:
terms = tf_idf.get_feature_names_out() 

# sum tfidf frequency of each term through documents
sums = tfidf_mat.sum(axis=0)

# connecting term to its sums frequency
datas = []
for col, term in enumerate(terms):
    datas.append((term, sums[col] ))

ranking = pd.DataFrame(datas, columns=['term','rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
2086,hasil,24.314880
3617,metode,23.106305
5443,sistem,23.092523
5886,teliti,22.589087
3879,nilai,21.270546
...,...,...
1978,glukoma,0.147670
3123,lce,0.147670
3985,onh,0.147670
4001,ophthalmologis,0.147670


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize

max_features = 10000

# calc TF vector
cvect = CountVectorizer(max_features=max_features)
TF_vector = cvect.fit_transform(datap["text_join"])

# normalize TF vector
normalized_TF_vector = normalize(TF_vector, norm='l1', axis=1)

# calc IDF
tfidf = TfidfVectorizer(max_features=max_features, smooth_idf=False)
tfs = tfidf.fit_transform(datap["text_join"])
IDF_vector = tfidf.idf_

# hitung TF x IDF sehingga dihasilkan TFIDF matrix / vector
tfidf_mat = normalized_TF_vector.multiply(IDF_vector).toarray()

In [26]:
# hasil dari tf-idf 
dfbtf= pd.DataFrame(data=tfidf_mat,columns=[terms])
dfbtf

,aalysis,aam,ab,abad,abadi,abai,abdi,ability,abjad,absah,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## PCA

In [27]:
# proses pca menjadi 150 fitur
from sklearn.decomposition import PCA
pca_abstrak = PCA(n_components=150)
principalComponents_abstrak = pca_abstrak.fit_transform(dfbtf)
pca_abstrak = pd.DataFrame(principalComponents_abstrak)
pca_abstrak

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.010807,-0.067902,-0.014043,-0.029752,-0.036381,0.039716,-0.044379,-0.046355,-0.002975,-0.010948,...,0.008849,0.045377,-0.028546,0.056910,0.006408,0.003320,-0.014917,0.027204,-0.004645,0.007131
1,0.010503,-0.041949,-0.026737,-0.070804,-0.025507,0.085415,-0.037616,0.047586,0.009091,-0.008978,...,0.015823,0.104098,-0.053677,-0.041240,-0.051224,-0.035444,0.050340,0.076712,0.012594,0.069876
2,0.016411,-0.037037,-0.063963,-0.057730,-0.089253,0.116742,0.139919,-0.154654,0.152187,0.029949,...,-0.000419,0.022707,-0.009283,0.053851,-0.002963,-0.000672,0.016167,-0.007141,-0.056236,-0.036546
3,0.016487,-0.045988,-0.058216,0.045278,-0.003275,0.030264,-0.002741,-0.035236,0.043659,0.381641,...,-0.028474,0.000392,-0.029506,0.020934,-0.003503,0.009016,-0.021337,0.001316,0.000506,-0.004886
4,0.061963,0.024015,0.019398,0.002925,-0.010613,0.154533,-0.045083,0.090644,0.015553,-0.003554,...,0.033111,0.001062,0.022410,-0.000530,0.038111,0.010169,-0.019801,-0.014889,0.004670,0.024683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,-0.014959,-0.018901,0.015667,-0.025576,-0.022289,-0.027296,0.030206,0.029163,-0.013907,0.014885,...,-0.006386,-0.003986,-0.012574,0.017287,-0.001891,0.012007,-0.003073,0.020928,0.016804,0.023638
801,-0.013179,-0.013911,-0.026699,-0.018075,-0.046292,0.030085,0.116091,-0.076824,0.084378,0.026757,...,-0.015134,0.032048,0.053334,-0.010069,0.045842,0.029364,-0.003706,0.025498,-0.021497,-0.057214
802,-0.107663,0.060590,-0.008556,0.018382,-0.027179,-0.014261,0.034473,0.011080,-0.018101,0.004872,...,-0.000104,-0.017490,0.005445,-0.004993,-0.017042,0.050612,0.008527,-0.014655,0.028720,-0.032244
803,-0.072869,0.034535,0.002877,-0.001252,-0.009726,-0.010885,0.040183,0.039794,-0.010442,-0.027320,...,-0.012310,0.005399,0.026647,0.035296,-0.034263,0.016491,0.054084,-0.023913,0.033639,-0.003647


### Split Data

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
# split tanpa PCA
X=dfbtf
y=datap['Kategori']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [30]:
# split  PCA
pca_X=pca_abstrak
pca_y=datap['Kategori']

pca_X_train,pca_X_test,pca_y_train,pca_y_test=train_test_split(pca_X,pca_y,test_size=0.3,random_state=0)

## Naive bayes

In [31]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB


In [32]:
# HASIL AKURASI TANPA PCA
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
YY_pred = gaussian.predict(X_test)
accuracy_nb=round(accuracy_score(y_test,YY_pred)* 100, 2)
print(accuracy_nb)


73.97


In [34]:
# HASIL AKURASI dengan PCA
pca_gaussian = GaussianNB()
pca_gaussian.fit(pca_X_train, pca_y_train)
pca_Y_pred = pca_gaussian.predict(pca_X_test)
accuracy_nb_pca=round(accuracy_score(pca_y_test,pca_Y_pred)* 100, 2)
print(accuracy_nb_pca)

75.21




*   hasil metode naive bayes untuk data yang tanpa pca 73.97 % sedangkan data yang menggunakan pca adalah 75.21 % 
*   lebih tinggi menggunakan pca



## KNN

In [37]:
# HASIL AKURASI TANPA PCA
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train,y_train)

y_pred = knn_clf.predict(X_test)

accuracy_KNN=round(accuracy_score(y_test,y_pred)*100,2)
print(accuracy_KNN)

76.03


In [38]:
# HASIL AKURASI TANPA PCA
pca_knn_clf = KNeighborsClassifier(n_neighbors=3)
pca_knn_clf.fit(pca_X_train,pca_y_train)

pca_y_pred = pca_knn_clf.predict(pca_X_test)

accuracy_KNN_pca=round(accuracy_score(pca_y_test,pca_y_pred)*100,2)
print(accuracy_KNN_pca)

81.4




*   hasil knn dengan nilai k = 3 untuk data tanpa pca adalah 76.03 % sedangkan dengan menggunakna pca adalah 81.4 % 
*   hasil akurasinya lebih bagus menggunakan pca



### pencarian nilai k terbaik untuk metode knn

In [44]:
from pandas.core.missing import mask_missing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# membuat loopingan untuk mengecek score data dari data 1 sampai 500
data = [] #menyimpan hasil score tanpa pca
data_pca = [] #menyimpan hasil score dengan pca
data1 = [] #menyimpan data yang memiliki score terbesar tanpa pca
data1_pca = [] #menyimpan data yang memiliki score terbesar dengan pca
for i in range(1,500):
  knn = KNeighborsClassifier(n_neighbors = i)
  # knn tanpa pca
  knn.fit(X_train,y_train)
  y_pred = knn.predict(X_test)
  accuracy_KNN=round(accuracy_score(y_test,y_pred)*100,2)
  data.append(accuracy_KNN)

  #knn dengan pca
  knn.fit(pca_X_train,pca_y_train)
  pca_y_pred = knn.predict(pca_X_test)
  accuracy_KNN_pca=round(accuracy_score(pca_y_test,pca_y_pred)*100,2)
  data_pca.append(accuracy_KNN_pca)

maksimum = np.max(data)
maksimum_pca = np.max(data_pca)

for i in range (len(data)):

  if data[i] == maksimum:
    data1.append(i+1)
  if data_pca[i] == maksimum_pca:
    data1_pca.append(i+1)

print ('nilai akurasi maksimum untuk data tanpa pca adalah : ',maksimum)
print ("nilai k maksimum untuk data tanpa pca adalah nilai k ke- ",data1)
print ('nilai akurasi maksimum untuk data menggunakan pca adalah : ',maksimum_pca)
print ("nilai k maksimum untuk data menggunakan pca adalah nilai k ke-",data1)

nilai akurasi maksimum untuk data tanpa pca adalah :  80.58
nilai k maksimum untuk data tanpa pca adalah nilai k ke-  [9]
nilai akurasi maksimum untuk data menggunakan pca adalah :  85.95
nilai k maksimum untuk data menggunakan pca adalah nilai k ke- [9]




*   nilai k terbaik untuk data tanpa pca adalah nilai k ke-9 dengan akurasi sebesar 80.58 % sedangkan nilai k terbaik untuk data dengan menggunakan pca adalah nilai k ke- 9 dengan akurasi sebesar 85.95 %
*   sehingga dari hasil di atas dengan metode knn lebih baik menggunakan pca dengan menggunakan nilai k 9 dengan akurasi maksimumnya adalah 85.95



## Pencarian nilai PCA terbaik

In [45]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [48]:
data_nb = [] #menyimpan hasil score naive bayes
data_knn = [] #menyimpan hasil score knn
data1 = [] #menyimpan data akurasi yang memiliki score terbesar
data2 = [] 
for i in range(1,500):
  pca_abstrak = PCA(n_components=i)
  principalComponents_abstrak = pca_abstrak.fit_transform(dfbtf)
  pca_abstrak1 = pd.DataFrame(principalComponents_abstrak)

  # split  PCA
  pca_X=pca_abstrak1
  pca_y=datap['Kategori']
  pca_X_train,pca_X_test,pca_y_train,pca_y_test=train_test_split(pca_X,pca_y,test_size=0.3,random_state=0)

  # HASIL AKURASI dengan PCA
  # naive bayes
  gaussian = GaussianNB()
  gaussian.fit(pca_X_train, pca_y_train)
  pca_Y_pred = gaussian.predict(pca_X_test)
  accuracy_nb_pca2=round(accuracy_score(pca_y_test,pca_Y_pred)* 100, 2)
  data_nb.append(accuracy_nb_pca2)
  # knn
  knn = KNeighborsClassifier(n_neighbors = 9)
  knn.fit(pca_X_train,pca_y_train)
  pca_y_pred = knn.predict(pca_X_test)
  accuracy_KNN_pca=round(accuracy_score(pca_y_test,pca_y_pred)*100,2)
  data_knn.append(accuracy_KNN_pca)

maksimum_nb = np.max(data_nb)
maksimum_knn = np.max(data_knn)

for i in range (len(data)):
  if data_nb[i] == maksimum_nb:
    data1.append(i+1)
  if data_knn[i] == maksimum_knn:
    data2.append(i+1)
print ('nilai akurasi naive bayes maksimum yang di dapatkan : ',maksimum_nb)
print ("nilai pca naive bayes maksimum",data1)
print ("jumlah nilai pca naive bayes maksimum",len(data1))
print ('nilai akurasi knn maksimum yang di dapatkan : ',maksimum_knn)
print ("jumlah nilai pca naive bayes maksimum",len(data2))

nilai akurasi naive bayes maksimum yang di dapatkan :  78.93
nilai pca naive bayes maksimum [9, 133]
jumlah nilai pca naive bayes maksimum 2
nilai akurasi knn maksimum yang di dapatkan :  88.02
jumlah nilai pca naive bayes maksimum 1


In [51]:
print ("nilai pca knn maksimum",data2)

nilai pca knn maksimum [111]


## Kesimpulan

*   dari hasil pencarian nilai yang terbaik dari 500 kali percobaan hasilnya akurasi terbaik untuk metode knn adalah 88.02  % dan nilai akurasi terbaik untuk naive bayes adalah 78.98
*   sedangkan nilai pca yang memiliki nilai akurasi tertinggi untuk naive bayes ada pada pca ke 9 dan 133 dengan nilai akurasi 78.93 % dan untuk knn nilai pca tebaik adalah pca ke 111 yang menghasilkan akurasi 88.02 %
* sehingga dari kedua metode yang terbaik adalah dengan menggunakan pca 111 untuk metode knn dengan nilai k = 9 yang akan menghasilkan nilai akurasi sebesar 88.02 %



- referensi:

- https://yunusmuhammad007.medium.com/text-preprocessing-menggunakan-pandas-nltk-dan-sastrawi-untuk-large-dataset-5fb3c0a88571


- https://yunusmuhammad007.medium.com/tf-idf-term-frequency-inverse-document-frequency-representasi-vector-data-text-2a4eff56cda